In [4]:
import os
import pandas as pd

# Function to remove numericals and spaces at the front and back
def clean_string(s):
    return s.str.replace(r'^[0-9\s]+|[0-9\s]+$', '')

# List of all the airlines folders and corresponding csv names
airline_folders = ['1 Ryanair', '2 Lufthansa', '3 International Airlines Group', '4 AirFrance-KLM',
                   '5 Turkish Airlines', '6 EasyJet', '7 WizzAir', '8 Aeroflot Group', '9 Pegasus Airlines', '10 SAS Group']

airline_csv_names = ['ryanair', 'lufthansa', 'iag', 'airfrance_klm',
                   'turkish_airlines', 'easyjet', 'wizzair', 'aeroflot_group', 'pegasus_airlines', 'sas_scandinavian_airlines']

# Output folder
output_folder = 'all_airlines'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Define the types of datasets
dataset_types = ['reviews', 'reviews_sentiment_analysis', 'reviews_sentiment_analysis_lemmatized', 'reviews_routesummary']

# Dictionary to store DataFrames
all_dfs = {}

# Loop through each airline folder and its corresponding csv name
for airline, airline_name in zip(airline_folders, airline_csv_names):
    airline_folder_path = os.path.join(airline, 'data')

    # Loop through each dataset type
    for dataset_type in dataset_types:
        # Read the current dataset type
        current_df = pd.read_csv(os.path.join(airline_folder_path, f'{airline_name}_{dataset_type}.csv'))
        current_df['airline'] = airline

        # Apply cleaning function
        current_df['airline'] = clean_string(current_df['airline'])

        # Reset index and drop 'Unnamed: 0'
        current_df.reset_index(inplace=True, drop=False)
        current_df.drop('Unnamed: 0', axis=1, inplace=True)

        # Store the DataFrame in the dictionary
        key = f'all_airlines_{dataset_type}'
        if key in all_dfs:
            all_dfs[key].append(current_df)
        else:
            all_dfs[key] = [current_df]

# Concatenate DataFrames for each dataset type
for key, dfs in all_dfs.items():
    all_dfs[key] = pd.concat(dfs)

# Save DataFrames to CSV
for key, df in all_dfs.items():
    df.to_csv(os.path.join(output_folder, f'{key}.csv'), index=False)


/var/folders/9j/r5th0fxd26341bbn6qkwvlww0000gn/T/ipykernel_9822/2687544457.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(r'^[0-9\s]+|[0-9\s]+$', '')
/var/folders/9j/r5th0fxd26341bbn6qkwvlww0000gn/T/ipykernel_9822/2687544457.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(r'^[0-9\s]+|[0-9\s]+$', '')
/var/folders/9j/r5th0fxd26341bbn6qkwvlww0000gn/T/ipykernel_9822/2687544457.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(r'^[0-9\s]+|[0-9\s]+$', '')
/var/folders/9j/r5th0fxd26341bbn6qkwvlww0000gn/T/ipykernel_9822/2687544457.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(r'^[0-9\s]+|[0-9\s]+$', '')
/var/folders/9j/r5th0fxd26341bbn6qkwvlww0000gn/T/ipykernel_9822/2687544457.py:6: FutureWarning: 

In [5]:
# Read the appropiate
all_df = pd.read_csv(os.path.join('all_airlines', 'all_airlines_reviews_sentiment_analysis_lemmatized.csv'))
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10984 entries, 0 to 10983
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      10984 non-null  int64  
 1   authors                    10984 non-null  object 
 2   countries                  10984 non-null  object 
 3   date                       10984 non-null  object 
 4   rating                     10984 non-null  int64  
 5   reviews                    10984 non-null  object 
 6   aircraft                   4608 non-null   object 
 7   traveller type             10982 non-null  object 
 8   seat type                  10984 non-null  object 
 9   route                      10950 non-null  object 
 10  date flown                 10983 non-null  object 
 11  seat comfort               10129 non-null  float64
 12  cabin staff service        10076 non-null  float64
 13  food & beverages           7522 non-null   flo

In [6]:
# Topic Modelling

from gensim import corpora, models
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
# Topic Modelling Code for All reviews

# List of all the airlines folders and corresponding csv names
airline_folders = ['1 Ryanair', '2 Lufthansa', '3 International Airlines Group', '4 AirFrance-KLM',
                   '5 Turkish Airlines', '6 EasyJet', '7 WizzAir', '8 Aeroflot Group', '9 Pegasus Airlines', '10 SAS Group']

airline_csv_names = ['ryanair', 'lufthansa', 'iag', 'airfrance_klm',
                   'turkish_airlines', 'easyjet', 'wizzair', 'aeroflot_group', 'pegasus_airlines', 'sas_scandinavian_airlines']

# Define the types of datasets
filename = 'reviews_sentiment_analysis_lemmatized'


# Loop through each airline folder and its corresponding csv name
for airline, airline_name in zip(airline_folders, airline_csv_names):
    airline_folder_path = os.path.join(airline, 'data')

    # Read the current dataset type
    current_df = pd.read_csv(os.path.join(airline_folder_path, f'{airline_name}_{filename}.csv'))

    # # Apply cleaning function
    # current_df['airline'] = clean_string(current_df['airline'])

    # # Reset index and drop 'Unnamed: 0'
    # current_df.reset_index(inplace=True, drop=False)
    # current_df.drop('Unnamed: 0', axis=1, inplace=True)

    # Create a document-term matrix using TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
    tfidf_lemma = tfidf_vectorizer.fit_transform(current_df['lemmatized reviews'])

    # Perform LDA topic modeling
    num_topics = 1
    lda_lemma = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_lemma.fit(tfidf_lemma)

    # Display the topics and associated words
    def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
            print(f"Topic #{topic_idx + 1}:")
            print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
            print()

    n_top_words = 10
    tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
    print(f'{airline}: LDA with lemmatized reviews')
    print_top_words(lda_lemma, tfidf_feature_names, n_top_words)

1 Ryanair: LDA with lemmatized reviews
Topic #1:
pay time check seat get fly board hour airport staff

2 Lufthansa: LDA with lemmatized reviews
Topic #1:
seat frankfurt service time get good hour class business food

3 International Airlines Group: LDA with lemmatized reviews
Topic #1:
seat ba service get time london good hour crew food

4 AirFrance-KLM: LDA with lemmatized reviews
Topic #1:
seat amsterdam service time get fly good hour crew food

5 Turkish Airlines: LDA with lemmatized reviews
Topic #1:
istanbul seat service hour time ticket get airport staff customer

6 EasyJet: LDA with lemmatized reviews
Topic #1:
time delay bag board hour get staff seat us fly

7 WizzAir: LDA with lemmatized reviews
Topic #1:
pay air check time airport fly seat get hour service

8 Aeroflot Group: LDA with lemmatized reviews
Topic #1:
moscow good seat time service hour food get staff fly

9 Pegasus Airlines: LDA with lemmatized reviews
Topic #1:
istanbul time pay hour staff get seat delay ticket us

In [8]:
# Topic Modelling Code for Non-Recommended Reviews

# List of all the airlines folders and corresponding csv names
airline_folders = ['1 Ryanair', '2 Lufthansa', '3 International Airlines Group', '4 AirFrance-KLM',
                   '5 Turkish Airlines', '6 EasyJet', '7 WizzAir', '8 Aeroflot Group', '9 Pegasus Airlines', '10 SAS Group']

airline_csv_names = ['ryanair', 'lufthansa', 'iag', 'airfrance_klm',
                   'turkish_airlines', 'easyjet', 'wizzair', 'aeroflot_group', 'pegasus_airlines', 'sas_scandinavian_airlines']

# Define the types of datasets
filename = 'reviews_sentiment_analysis_lemmatized'


# Loop through each airline folder and its corresponding csv name
for airline, airline_name in zip(airline_folders, airline_csv_names):
    airline_folder_path = os.path.join(airline, 'data')

    # Read the current dataset type
    current_df = pd.read_csv(os.path.join(airline_folder_path, f'{airline_name}_{filename}.csv'))

    # Only use the non recommended columns
    current_df_neg_reviews = current_df[current_df['recommended'] == 'no']

    # Create a document-term matrix using TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
    tfidf_lemma = tfidf_vectorizer.fit_transform(current_df_neg_reviews['lemmatized reviews'])

    # Perform LDA topic modeling
    num_topics = 1
    lda_lemma = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_lemma.fit(tfidf_lemma)

    # Display the topics and associated words
    def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
            print(f"Topic #{topic_idx + 1}:")
            print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
            print()

    n_top_words = 10
    tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
    print(f'{airline}: LDA with lemmatized non recommended reviews')
    print_top_words(lda_lemma, tfidf_feature_names, n_top_words)

1 Ryanair: LDA with lemmatized non recommended reviews
Topic #1:
pay check get fly time seat hour us bag airport

2 Lufthansa: LDA with lemmatized non recommended reviews
Topic #1:
seat service frankfurt time get hour customer fly us airport

3 International Airlines Group: LDA with lemmatized non recommended reviews
Topic #1:
seat ba service get hour london class time us fly

4 AirFrance-KLM: LDA with lemmatized non recommended reviews
Topic #1:
seat get service amsterdam luggage hour time us customer fly

5 Turkish Airlines: LDA with lemmatized non recommended reviews
Topic #1:
istanbul seat ticket hour service get time us customer airport

6 EasyJet: LDA with lemmatized non recommended reviews
Topic #1:
delay bag hour time get pay us staff fly luggage

7 WizzAir: LDA with lemmatized non recommended reviews
Topic #1:
pay check air airport get hour fly time service us

8 Aeroflot Group: LDA with lemmatized non recommended reviews
Topic #1:
moscow hour get time luggage seat refund airp

In [9]:
# Read the appropiate
all_df = pd.read_csv(os.path.join('all_airlines', 'all_airlines_reviews_sentiment_analysis_lemmatized.csv'))

# Only use the non recommended columns
df_neg_reviews = all_df[all_df['recommended'] == 'no']

# Create a document-term matrix using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf_lemma = tfidf_vectorizer.fit_transform(df_neg_reviews['lemmatized reviews'])

# Perform LDA topic modeling
num_topics = 1
lda_lemma = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_lemma.fit(tfidf_lemma)

# Display the topics and associated words
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()

n_top_words = 10
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print('All Airlines: LDA with lemmatized non recommended reviews')
print_top_words(lda_lemma, tfidf_feature_names, n_top_words)

All Airlines: LDA with lemmatized non recommended reviews
Topic #1:
seat get hour service time pay us fly customer check



In [10]:
# Read the appropiate
all_df = pd.read_csv(os.path.join('all_airlines', 'all_airlines_reviews_sentiment_analysis_lemmatized.csv'))

# Create a document-term matrix using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf_lemma = tfidf_vectorizer.fit_transform(all_df['lemmatized reviews'])

# Perform LDA topic modeling
num_topics = 1
lda_lemma = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_lemma.fit(tfidf_lemma)

# Display the topics and associated words
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()

n_top_words = 10
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print('All Airlines: LDA with lemmatized recommended reviews')
print_top_words(lda_lemma, tfidf_feature_names, n_top_words)

All Airlines: LDA with lemmatized recommended reviews
Topic #1:
seat time service get hour fly pay staff us good



In [11]:
# Read the appropiate
all_df = pd.read_csv(os.path.join('all_airlines', 'all_airlines_reviews_sentiment_analysis_lemmatized.csv'))

# Only use the recommended columns
df_neg_reviews = all_df[all_df['recommended'] == 'yes']

# Create a document-term matrix using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf_lemma = tfidf_vectorizer.fit_transform(df_neg_reviews['lemmatized reviews'])

# Perform LDA topic modeling
num_topics = 1
lda_lemma = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_lemma.fit(tfidf_lemma)

# Display the topics and associated words
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()

n_top_words = 10
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print('All Airlines: LDA with lemmatized non recommended reviews')
print_top_words(lda_lemma, tfidf_feature_names, n_top_words)

All Airlines: LDA with lemmatized non recommended reviews
Topic #1:
seat good time crew service cabin board food fly staff

